In [1]:
import pandas as pd
import numpy as np
from radon.visitors import ComplexityVisitor
from radon.complexity import cc_rank
from radon.metrics import h_visit, mi_visit
from radon.raw import analyze
import autopep8

# Загрузка данных по корректному коду

In [21]:
import pandas as pd
path_to_data = '/home/kks/zephyr_data/stable_code/'
source_code_data = pd.read_pickle(path_to_data + 'correct_source_code_train.pickle')

In [22]:
source_code_data['before_merge'].apply(checkExtraSpaces).sum() / source_code_data.shape[0]

0.0

In [19]:
import re

def checkExtraSpaces(snippet):
    number_of_spaces_to_remove = re.search('\S', snippet.split('\n')[0]).start(0)
    if number_of_spaces_to_remove > 0:
        return True

def removeExtraSpaces(snippet):
    number_of_spaces_to_remove = re.search('\S', snippet.split('\n')[0]).start(0)
    return '\n'.join([snippet_line[number_of_spaces_to_remove:] 
                      for snippet_line in snippet.split('\n')])


def refineSnippet(snippet):
    try:
        ComplexityVisitor.from_code(snippet)
        return snippet
    except:
        try:
            snippet_with_removed_extra_spaces = removeExtraSpaces(snippet)
            ComplexityVisitor.from_code(snippet_with_removed_extra_spaces)
            return snippet_with_removed_extra_spaces
        except:
            return np.nan

# Функции вычисления метрик

In [7]:
def computeCyclomaticComplexity(snippet):
    return ComplexityVisitor.from_code(snippet).functions_complexity


def isFunction(snippet):
    return not ComplexityVisitor.from_code(snippet).functions[0].is_method


def computeHastadMetrics(snippet):
    hastad_metrics = h_visit(snippet).total
    return {metric:getattr(hastad_metrics, metric) for metric in dir(hastad_metrics) 
            if (not metric.startswith('_')) and ('index' not in metric) and ('count' not in metric)}


def computeRawMetrics(snippet):
    raw_metrics = analyze(snippet)
    return {metric:getattr(raw_metrics, metric) for metric in dir(raw_metrics) 
            if (not metric.startswith('_')) and ('index' not in metric) and ('count' not in metric)}


def computeMIindex(snippet):
    return mi_visit(snippet, multi=False)

# Цикломатическая сложность

In [8]:
source_code_data['cc_before'] = source_code_data['before_merge'].apply(computeCyclomaticComplexity)
source_code_data['is function_before'] = source_code_data['before_merge'].apply(isFunction)
source_code_data['cc_rank_before'] = source_code_data['cc_before'].apply(cc_rank)

# Метрики Хастада

In [9]:
hastad_metrics_before = source_code_data['before_merge'].apply(computeHastadMetrics)

In [10]:
for col in hastad_metrics_before.loc[0].keys():
    source_code_data[col + '_before'] = hastad_metrics_before.apply(lambda x: x[col])

# Сырые метрики

In [11]:
raw_metrics_before = source_code_data['before_merge'].apply(computeRawMetrics)

In [12]:
for col in raw_metrics_before.loc[0].keys():
    source_code_data[col + '_before'] = raw_metrics_before.apply(lambda x: x[col])

# Индекс

In [13]:
source_code_data['MI_before'] = source_code_data['before_merge'].apply(computeMIindex)

In [14]:
metric_names = ['cc_before', 'is function_before',
                'cc_rank_before', 'N1_before', 'N2_before',
                'bugs_before', 'calculated_length_before',
                'difficulty_before',
                'effort_before', 'h1_before', 'h2_before',
                'length_before', 'time_before',
                'vocabulary_before', 'volume_before',
                'blank_before', 'comments_before',
                'lloc_before', 'loc_before',
                'multi_before', 'single_comments_before',
                'sloc_before', 'MI_before']

In [15]:
source_code_data.to_pickle(path_to_data + correct_source_code_test.pickle)

,before_merge,repo_name,filename,function_name,syntax_correct,before_merge_without_docstrings,before_merge_docstrings,path_to_snippet_before_merge,cc_before,is function_before,...,vocabulary_before,volume_before,blank_before,comments_before,lloc_before,loc_before,multi_before,single_comments_before,sloc_before,MI_before
0,def user_follows():\n access_token = reques...,facebookarchive_python-instagram,sample_app.py,user_follows,True,def user_follows():\n access_token = reques...,[],stable_snippets_files/e56b911b5f2cf21fcc087a2d...,5,True,...,11,48.432043,0,1,19,20,0,1,19,72.880998
1,"def instrumented_stage(args, stage):\n """"""\...",ClangBuiltLinux_tc-build,build-llvm.py,instrumented_stage,True,"def instrumented_stage(args, stage):\n \n ...",[Returns true if we are using PGO and on stage...,stable_snippets_files/a45224293585c24940ddf7e3...,1,True,...,6,15.509775,0,0,3,8,6,0,2,80.986453
2,"def ev_collect(ev, level, obj):\n if isinst...",andikleen_pmu-tools,toplev.py,ev_collect,True,"def ev_collect(ev, level, obj):\n if isinst...",[],stable_snippets_files/759e99ada074389fef8c7f21...,12,True,...,20,125.335915,2,0,22,23,0,0,21,54.277149
3,def load_libc_args():\n # load libc functio...,hugsy_gef,gef.py,load_libc_args,True,def load_libc_args():\n \n if not get_ge...,[],stable_snippets_files/ab1a12c03f2ad8ead19ce1e5...,6,True,...,9,31.699250,7,2,27,36,0,2,27,72.953851
4,def get_curr_connects_count():\n global use...,alexbers_mtprotoproxy,mtprotoproxy.py,get_curr_connects_count,True,def get_curr_connects_count():\n global use...,[],stable_snippets_files/f9aad12115156a3b3162feda...,1,True,...,3,4.754888,1,0,6,7,0,0,6,78.015086
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
165,def get_junk_item(count=1):\n ret_junk = []...,AmazingAmpharos_OoT-Randomizer,ItemList.py,get_junk_item,True,def get_junk_item(count=1):\n ret_junk = []...,[],stable_snippets_files/aa1413a9b9b2c78a906c82d5...,1,True,...,0,0.000000,1,0,5,6,0,0,5,100.000000
166,"def is_idle(cpus, idle_keys):\n return all(...",andikleen_pmu-tools,toplev.py,is_idle,True,"def is_idle(cpus, idle_keys):\n return all(...",[],stable_snippets_files/33e1c61199b64686481abc05...,1,True,...,6,15.509775,0,0,2,2,0,0,2,84.827701
167,"def parole_crawl(path, processfn, debug_sgm_li...",gooofy_zamia-speech,parole.py,parole_crawl,True,"def parole_crawl(path, processfn, debug_sgm_li...",[],stable_snippets_files/54b52d91e62bf4271958db45...,7,True,...,18,104.248125,9,0,23,32,0,0,23,55.088754
168,def _set_version(version):\n if not version...,ashishb_adb-enhanced,release.py,_set_version,True,def _set_version(version):\n if not version...,[],stable_snippets_files/0487d88e2b8f7937cf482f9b...,2,True,...,8,30.000000,0,0,5,5,0,0,5,74.006362
